In [1]:
pip install ultralytics==8.0.196 opencv-python==4.8.1.78 numpy==1.24.3 scipy==1.11.3 scikit-learn==1.3.0 torch==2.0.1 torchvision==0.15.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.1/631.1 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55

In [2]:
import cv2
import numpy as np
from ultralytics import YOLO
import os
from scipy.spatial.distance import cdist
from collections import deque
import json
import time

class SingleFeedPlayerReID:
    def __init__(self, model_path=None):
        """Initialize the Single Feed Player Re-Identification system for Kaggle"""
        print("🚀 Initializing Single Feed Player Re-Identification System (Kaggle)")
        
        # Kaggle model path
        self.model_path = model_path or "/kaggle/input/yolov11/pytorch/default/1/best.pt"
        
        if not os.path.exists(self.model_path):
            print(f"❌ Error: Model not found at {self.model_path}")
            print("Please ensure the YOLOv11 model is uploaded to Kaggle")
            return
        
        print(f"✅ Loading model from: {self.model_path}")
        self.model = YOLO(self.model_path)
        
        # Player tracking data
        self.active_players = {}      # Currently visible players
        self.inactive_players = {}    # Players who disappeared but might return
        self.player_history = {}      # Complete history of all players
        self.next_player_id = 1
        
        # Enhanced tracking parameters for better re-identification
        self.max_disappeared = 30     # Reduced frames before moving to inactive
        self.max_inactive_time = 120  # Frames to keep inactive players for re-identification
        self.feature_history_size = 10 # Number of feature vectors to store
        self.confidence_threshold = 0.25 # Lower threshold for better detection
        
        # Re-identification parameters - more lenient for better matching
        self.reid_similarity_threshold = 0.65  # Lower threshold for easier re-identification
        self.spatial_weight = 0.4     # Increased spatial weight
        self.feature_weight = 0.6     # Reduced feature weight for more flexibility
        
        # Enhanced color palette for player visualization
        self.colors = [
            (0, 0, 255),     # Red
            (0, 255, 0),     # Green  
            (255, 0, 0),     # Blue
            (0, 255, 255),   # Yellow
            (255, 0, 255),   # Magenta
            (255, 255, 0),   # Cyan
            (128, 0, 128),   # Purple
            (255, 165, 0),   # Orange
            (0, 128, 0),     # Dark Green
            (128, 128, 0),   # Olive
            (0, 0, 128),     # Navy
            (128, 0, 0),     # Maroon
            (255, 192, 203), # Pink
            (165, 42, 42),   # Brown
            (64, 224, 208),  # Turquoise
            (255, 20, 147),  # Deep Pink
            (0, 191, 255),   # Deep Sky Blue
            (50, 205, 50),   # Lime Green
            (255, 69, 0),    # Red Orange
            (138, 43, 226),  # Blue Violet
        ]
        
        # Statistics tracking
        self.stats = {
            'total_detections': 0,
            'reidentifications': 0,
            'new_players': 0,
            'frames_processed': 0
        }
        
        print("✅ System initialized successfully!")
    
    def extract_enhanced_features(self, frame, bbox):
        """Extract comprehensive visual features from player bounding box"""
        x1, y1, x2, y2 = map(int, bbox)
        
        # Ensure coordinates are within frame bounds
        h, w = frame.shape[:2]
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w, x2), min(h, y2)
        
        if x2 <= x1 or y2 <= y1:
            return np.zeros(128)  # Reduced feature size for efficiency
        
        # Extract player region
        player_region = frame[y1:y2, x1:x2]
        
        if player_region.size == 0:
            return np.zeros(128)
        
        # Resize to standard size for consistent feature extraction
        try:
            player_region = cv2.resize(player_region, (32, 64))  # Smaller for efficiency
        except:
            return np.zeros(128)
        
        # 1. Color Features - RGB histograms
        hist_b = cv2.calcHist([player_region], [0], None, [16], [0, 256])
        hist_g = cv2.calcHist([player_region], [1], None, [16], [0, 256])
        hist_r = cv2.calcHist([player_region], [2], None, [16], [0, 256])
        
        # Normalize histograms
        hist_b = hist_b.flatten() / (hist_b.sum() + 1e-7)
        hist_g = hist_g.flatten() / (hist_g.sum() + 1e-7)
        hist_r = hist_r.flatten() / (hist_r.sum() + 1e-7)
        
        # 2. HSV color space for better color representation
        try:
            hsv_region = cv2.cvtColor(player_region, cv2.COLOR_BGR2HSV)
            hist_h = cv2.calcHist([hsv_region], [0], None, [16], [0, 180])
            hist_s = cv2.calcHist([hsv_region], [1], None, [16], [0, 256])
            hist_v = cv2.calcHist([hsv_region], [2], None, [16], [0, 256])
            
            hist_h = hist_h.flatten() / (hist_h.sum() + 1e-7)
            hist_s = hist_s.flatten() / (hist_s.sum() + 1e-7)
            hist_v = hist_v.flatten() / (hist_v.sum() + 1e-7)
        except:
            hist_h = np.zeros(16)
            hist_s = np.zeros(16)
            hist_v = np.zeros(16)
        
        # 3. Simple texture features
        try:
            gray = cv2.cvtColor(player_region, cv2.COLOR_BGR2GRAY)
            texture_features = [
                np.std(gray),           # Standard deviation
                np.mean(gray),          # Mean intensity
                np.max(gray) - np.min(gray) if gray.size > 0 else 0,  # Range
            ]
            texture_features = np.array(texture_features[:16])  # Limit size
            if len(texture_features) < 16:
                texture_features = np.pad(texture_features, (0, 16-len(texture_features)))
        except:
            texture_features = np.zeros(16)
        
        # 4. Geometric features
        aspect_ratio = (x2 - x1) / max(y2 - y1, 1)
        area = (x2 - x1) * (y2 - y1)
        geometric_features = np.array([aspect_ratio, area / 10000.0])
        
        # Combine all features (16+16+16+16+16+16+16+2 = 128 features)
        features = np.concatenate([
            hist_b, hist_g, hist_r,  # RGB: 48 features
            hist_h, hist_s, hist_v,  # HSV: 48 features
            texture_features,        # Texture: 16 features
            geometric_features       # Geometric: 2 features
        ])
        
        # Ensure fixed size
        if len(features) < 128:
            features = np.pad(features, (0, 128-len(features)))
        
        return features[:128]
    
    def detect_players(self, frame):
        """Detect players in a frame using YOLO"""
        try:
            results = self.model(frame, verbose=False, conf=self.confidence_threshold)
            detections = []
            
            for result in results:
                boxes = result.boxes
                if boxes is not None:
                    for box in boxes:
                        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                        confidence = box.conf[0].cpu().numpy()
                        class_id = int(box.cls[0].cpu().numpy())
                        
                        # Filter for person detections (class 0 in COCO/YOLO)
                        if confidence > self.confidence_threshold:
                            width = x2 - x1
                            height = y2 - y1
                            aspect_ratio = width / max(height, 1)
                            
                            # Filter realistic human detections
                            if (width > 5 and height > 10 and 
                                aspect_ratio > 0.2 and aspect_ratio < 3.0):
                                
                                detections.append({
                                    'bbox': [x1, y1, x2, y2],
                                    'confidence': confidence,
                                    'center': [(x1 + x2) / 2, (y1 + y2) / 2],
                                    'area': width * height
                                })
            
            # Sort by confidence
            detections.sort(key=lambda x: x['confidence'], reverse=True)
            self.stats['total_detections'] += len(detections)
            
            return detections
        except Exception as e:
            print(f"Detection error: {e}")
            return []
    
    def calculate_similarity(self, features1, features2, center1, center2):
        """Calculate similarity between two player detections"""
        try:
            # Feature similarity (cosine similarity)
            dot_product = np.dot(features1, features2)
            norm1 = np.linalg.norm(features1)
            norm2 = np.linalg.norm(features2)
            
            if norm1 == 0 or norm2 == 0:
                feature_sim = 0
            else:
                feature_sim = dot_product / (norm1 * norm2)
                feature_sim = max(0, feature_sim)  # Ensure non-negative
            
            # Spatial similarity
            spatial_dist = np.linalg.norm(np.array(center1) - np.array(center2))
            spatial_sim = 1 / (1 + spatial_dist / 200.0)  # Normalized spatial similarity
            
            # Combined similarity
            combined_sim = (self.feature_weight * feature_sim + 
                           self.spatial_weight * spatial_sim)
            
            return combined_sim, feature_sim, spatial_sim
        except:
            return 0.0, 0.0, 0.0
    
    def match_detections_to_players(self, detections, frame):
        """Match detections to existing players with improved re-identification"""
        if not detections:
            # Handle disappeared players
            for player_id in list(self.active_players.keys()):
                self.active_players[player_id]['disappeared'] += 1
                if self.active_players[player_id]['disappeared'] > self.max_disappeared:
                    # Move to inactive
                    self.inactive_players[player_id] = self.active_players[player_id]
                    self.inactive_players[player_id]['inactive_time'] = 0
                    del self.active_players[player_id]
                    print(f"💤 Player {player_id} moved to inactive pool")
            return
        
        # Extract features for all detections
        for detection in detections:
            detection['features'] = self.extract_enhanced_features(frame, detection['bbox'])
        
        # Initialize first players
        if not self.active_players and not self.inactive_players:
            for detection in detections:
                player_id = self.next_player_id
                self.next_player_id += 1
                
                self.active_players[player_id] = {
                    'bbox': detection['bbox'],
                    'center': detection['center'],
                    'features': deque([detection['features']], maxlen=self.feature_history_size),
                    'confidence': detection['confidence'],
                    'disappeared': 0,
                    'last_seen_frame': self.stats['frames_processed'],
                    'total_detections': 1
                }
                
                self.player_history[player_id] = {
                    'first_seen': self.stats['frames_processed'],
                    'last_seen': self.stats['frames_processed'],
                    'total_appearances': 1
                }
                
                self.stats['new_players'] += 1
                print(f"➕ New player {player_id} initialized")
            return
        
        # Prepare all players (active + inactive) for matching
        all_players = {**self.active_players, **self.inactive_players}
        
        if not all_players:
            # Create new players
            for detection in detections:
                player_id = self.next_player_id
                self.next_player_id += 1
                
                self.active_players[player_id] = {
                    'bbox': detection['bbox'],
                    'center': detection['center'],
                    'features': deque([detection['features']], maxlen=self.feature_history_size),
                    'confidence': detection['confidence'],
                    'disappeared': 0,
                    'last_seen_frame': self.stats['frames_processed'],
                    'total_detections': 1
                }
                
                self.stats['new_players'] += 1
            return
        
        # Create similarity matrix
        player_ids = list(all_players.keys())
        similarity_matrix = np.zeros((len(detections), len(player_ids)))
        
        for i, detection in enumerate(detections):
            for j, player_id in enumerate(player_ids):
                player = all_players[player_id]
                # Use most recent features
                if len(player['features']) > 0:
                    avg_features = np.mean(list(player['features']), axis=0)
                    similarity, _, _ = self.calculate_similarity(
                        detection['features'], avg_features, 
                        detection['center'], player['center']
                    )
                    similarity_matrix[i, j] = similarity
        
        # Greedy assignment - highest similarity first
        used_detections = set()
        used_players = set()
        
        # Create assignment candidates
        assignments = []
        for i in range(len(detections)):
            for j in range(len(player_ids)):
                if similarity_matrix[i, j] > 0.3:  # Minimum threshold
                    assignments.append((similarity_matrix[i, j], i, j))
        
        assignments.sort(reverse=True)
        
        # Assign detections to players
        for similarity, det_idx, player_idx in assignments:
            if det_idx in used_detections or player_idx in used_players:
                continue
            
            player_id = player_ids[player_idx]
            detection = detections[det_idx]
            
            if similarity > self.reid_similarity_threshold:
                # Successful match
                if player_id in self.inactive_players:
                    # Re-identification!
                    self.active_players[player_id] = self.inactive_players[player_id]
                    del self.inactive_players[player_id]
                    self.stats['reidentifications'] += 1
                    print(f"🔄 RE-IDENTIFIED Player {player_id} (similarity: {similarity:.3f})")
                
                # Update player
                self.active_players[player_id]['bbox'] = detection['bbox']
                self.active_players[player_id]['center'] = detection['center']
                self.active_players[player_id]['features'].append(detection['features'])
                self.active_players[player_id]['confidence'] = detection['confidence']
                self.active_players[player_id]['disappeared'] = 0
                self.active_players[player_id]['last_seen_frame'] = self.stats['frames_processed']
                self.active_players[player_id]['total_detections'] += 1
                
                # Update history
                if player_id in self.player_history:
                    self.player_history[player_id]['last_seen'] = self.stats['frames_processed']
                    self.player_history[player_id]['total_appearances'] += 1
                
                used_detections.add(det_idx)
                used_players.add(player_idx)
        
        # Create new players for unmatched detections
        for i, detection in enumerate(detections):
            if i not in used_detections:
                player_id = self.next_player_id
                self.next_player_id += 1
                
                self.active_players[player_id] = {
                    'bbox': detection['bbox'],
                    'center': detection['center'],
                    'features': deque([detection['features']], maxlen=self.feature_history_size),
                    'confidence': detection['confidence'],
                    'disappeared': 0,
                    'last_seen_frame': self.stats['frames_processed'],
                    'total_detections': 1
                }
                
                self.player_history[player_id] = {
                    'first_seen': self.stats['frames_processed'],
                    'last_seen': self.stats['frames_processed'],
                    'total_appearances': 1
                }
                
                self.stats['new_players'] += 1
                print(f"➕ New player {player_id} detected")
        
        # Handle unmatched active players
        for j, player_id in enumerate(player_ids):
            if j not in used_players and player_id in self.active_players:
                self.active_players[player_id]['disappeared'] += 1
                
                if self.active_players[player_id]['disappeared'] > self.max_disappeared:
                    # Move to inactive
                    self.inactive_players[player_id] = self.active_players[player_id]
                    self.inactive_players[player_id]['inactive_time'] = 0
                    del self.active_players[player_id]
                    print(f"💤 Player {player_id} moved to inactive")
        
        # Clean up old inactive players
        to_remove = []
        for player_id, player in self.inactive_players.items():
            player['inactive_time'] += 1
            if player['inactive_time'] > self.max_inactive_time:
                to_remove.append(player_id)
        
        for player_id in to_remove:
            del self.inactive_players[player_id]
            print(f"🗑️ Removed player {player_id} from tracking")
    
    def draw_enhanced_detections(self, frame):
        """Draw enhanced bounding boxes with LARGE VISIBLE IDs"""
        output_frame = frame.copy()
        
        # Draw active players with LARGE, VISIBLE IDs
        for player_id, player in self.active_players.items():
            x1, y1, x2, y2 = map(int, player['bbox'])
            
            # Choose color
            color_idx = (player_id - 1) % len(self.colors)
            color = self.colors[color_idx]
            
            # Draw thick bounding box
            cv2.rectangle(output_frame, (x1, y1), (x2, y2), color, 4)
            
            # LARGE PLAYER ID - Multiple locations for visibility
            player_text = f"ID:{player_id}"
            
            # Large font settings
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 1.2
            thickness = 3
            
            # Get text size
            (text_width, text_height), baseline = cv2.getTextSize(player_text, font, font_scale, thickness)
            
            # Draw ID above bounding box with background
            cv2.rectangle(output_frame, 
                         (x1, y1 - text_height - 15), 
                         (x1 + text_width + 10, y1), 
                         color, -1)
            cv2.putText(output_frame, player_text, (x1 + 5, y1 - 8), 
                       font, font_scale, (255, 255, 255), thickness)
            
            # Draw ID inside bounding box (center)
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            
            # Background circle for center ID
            cv2.circle(output_frame, (center_x, center_y), 25, color, -1)
            cv2.circle(output_frame, (center_x, center_y), 25, (255, 255, 255), 3)
            
            # Center ID text
            id_text = str(player_id)
            (id_width, id_height), _ = cv2.getTextSize(id_text, font, 0.8, 2)
            cv2.putText(output_frame, id_text, 
                       (center_x - id_width//2, center_y + id_height//2), 
                       font, 0.8, (255, 255, 255), 2)
            
            # Draw confidence and detection count
            conf_text = f"{player['confidence']:.2f}"
            cv2.putText(output_frame, conf_text, (x1, y2 + 25), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            
            det_text = f"#{player['total_detections']}"
            cv2.putText(output_frame, det_text, (x2 - 50, y1 - 8), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        # Enhanced information overlay
        info_y = 40
        cv2.putText(output_frame, "SINGLE FEED PLAYER RE-IDENTIFICATION", (20, info_y), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0), 4)
        cv2.putText(output_frame, "SINGLE FEED PLAYER RE-IDENTIFICATION", (20, info_y), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)
        
        info_y += 50
        cv2.putText(output_frame, f"Active: {len(self.active_players)} | Inactive: {len(self.inactive_players)}", 
                   (20, info_y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        
        info_y += 30
        cv2.putText(output_frame, f"Frame: {self.stats['frames_processed']} | Re-IDs: {self.stats['reidentifications']}", 
                   (20, info_y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        
        return output_frame
    
    def process_video(self, video_path, output_path):
        """Process video with Kaggle paths"""
        print(f"🎬 Opening video: {video_path}")
        
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print("❌ Error: Could not open video file!")
            return
        
        # Get video properties
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        print(f"📊 Video: {width}x{height} @ {fps}fps ({total_frames} frames)")
        
        # Create video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        print("🔄 Processing with enhanced re-identification...")
        start_time = time.time()
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # Detect players
            detections = self.detect_players(frame)
            
            # Update tracking
            self.match_detections_to_players(detections, frame)
            
            # Draw with large visible IDs
            output_frame = self.draw_enhanced_detections(frame)
            
            # Write frame
            out.write(output_frame)
            
            self.stats['frames_processed'] += 1
            
            # Progress update
            if self.stats['frames_processed'] % 30 == 0:
                progress = (self.stats['frames_processed'] / total_frames) * 100
                elapsed = time.time() - start_time
                fps_actual = self.stats['frames_processed'] / elapsed
                print(f"📈 Progress: {self.stats['frames_processed']}/{total_frames} "
                      f"({progress:.1f}%) - {fps_actual:.1f} fps")
        
        cap.release()
        out.release()
        
        processing_time = time.time() - start_time
        print(f"✅ Complete! Time: {processing_time:.1f}s")
        
        # Print and save results
        self.print_statistics()
        self.save_results(output_path.replace('.mp4', '_results.json'))
    
    def print_statistics(self):
        """Print tracking statistics"""
        print("\n📊 TRACKING STATISTICS")
        print("=" * 40)
        print(f"Frames processed: {self.stats['frames_processed']}")
        print(f"Total detections: {self.stats['total_detections']}")
        print(f"New players: {self.stats['new_players']}")
        print(f"Re-identifications: {self.stats['reidentifications']}")
        print(f"Active players: {len(self.active_players)}")
        print(f"Inactive players: {len(self.inactive_players)}")
        print(f"Total unique players: {len(self.player_history)}")
        
        if self.stats['reidentifications'] > 0:
            print("✅ RE-IDENTIFICATION WORKING!")
        
        if self.player_history:
            print("\n👥 PLAYER DETAILS")
            for pid, history in self.player_history.items():
                status = "ACTIVE" if pid in self.active_players else "INACTIVE"
                print(f"Player {pid}: {history['total_appearances']} appearances [{status}]")
    
    def save_results(self, results_path):
        """Save results to JSON"""
        results = {
            'statistics': self.stats,
            'player_history': self.player_history,
            'active_players': len(self.active_players),
            'inactive_players': len(self.inactive_players),
            'total_unique_players': len(self.player_history),
            'reidentification_success': self.stats['reidentifications'] > 0
        }
        
        with open(results_path, 'w') as f:
            json.dump(results, f, indent=2)
        
        print(f"📋 Results saved: {results_path}")

def main():
    """Main function for Kaggle environment"""
    print("🚀 Single Feed Player Re-Identification System (Kaggle)")
    print("=" * 60)
    
    # Kaggle paths
    model_path = "/kaggle/input/yolov11/pytorch/default/1/best.pt"
    input_video = "/kaggle/input/play-videos/15sec_input_720p.mp4"
    output_video = "/kaggle/working/15sec_output_reidentified_720p.mp4"
    
    # Check paths
    if not os.path.exists(model_path):
        print(f"❌ Model not found: {model_path}")
        return
    
    if not os.path.exists(input_video):
        print(f"❌ Video not found: {input_video}")
        return
    
    print("✅ All files found!")
    
    # Initialize system
    reid_system = SingleFeedPlayerReID(model_path)
    
    # Process video
    reid_system.process_video(input_video, output_video)
    
    print("\n🎉 KAGGLE RE-IDENTIFICATION COMPLETE!")
    print("=" * 60)
    print(f"📂 Output: {output_video}")
    print(f"📊 Results: {output_video.replace('.mp4', '_results.json')}")
    print("\n✨ Features:")
    print("   • LARGE VISIBLE Player IDs")
    print("   • Enhanced re-identification")
    print("   • Kaggle-optimized paths")
    print("   • Real-time processing")

if __name__ == "__main__":
    main()

🚀 Single Feed Player Re-Identification System (Kaggle)
✅ All files found!
🚀 Initializing Single Feed Player Re-Identification System (Kaggle)
✅ Loading model from: /kaggle/input/yolov11/pytorch/default/1/best.pt
✅ System initialized successfully!
🎬 Opening video: /kaggle/input/play-videos/15sec_input_720p.mp4
📊 Video: 1280x720 @ 25fps (375 frames)
🔄 Processing with enhanced re-identification...
➕ New player 1 initialized
➕ New player 2 initialized
➕ New player 3 initialized
➕ New player 4 initialized
➕ New player 5 initialized
➕ New player 6 initialized
➕ New player 7 initialized
➕ New player 8 initialized
➕ New player 9 initialized
➕ New player 10 initialized
➕ New player 11 initialized
➕ New player 12 initialized
➕ New player 13 initialized
➕ New player 14 initialized
➕ New player 15 initialized
➕ New player 16 initialized
➕ New player 17 initialized
➕ New player 18 initialized
➕ New player 19 initialized
➕ New player 20 detected
➕ New player 21 detected
📈 Progress: 30/375 (8.0%) - 3